## Enqueue

There are several strategies for handling concurrent runs in your graph. This notebook covers how to use the `enqueue` option - please see the other how-to guides in the "Double Texting" directory to learn about the other methods.

First, let's import our required packages and instantiate our client, assistant, and thread.

In [ ]:
from langchain_core.messages import convert_to_messages
from langgraph_sdk import get_client

In [ ]:
client = get_client()
assistant = await client.assistants.create("agent")
thread = await client.threads.create()

In [ ]:
# this run will be interrupted
first_run = await client.runs.create(
    thread["thread_id"],
    assistant["assistant_id"],
    input={"messages": [{"role": "human", "content": "what's the weather in sf?"}]},
)

In [ ]:
second_run = await client.runs.create(
    thread["thread_id"],
    assistant["assistant_id"],
    input={"messages": [{"role": "human", "content": "what's the weather in nyc?"}]},
    multitask_strategy="enqueue",
)

Verify that the thread has data from both runs

In [ ]:
# wait until the second run completes
await client.runs.join(thread["thread_id"], second_run["run_id"])

In [ ]:
state = await client.threads.get_state(thread["thread_id"])

In [ ]:
for m in convert_to_messages(state["values"]["messages"]):
    m.pretty_print()

================================ Human Message =================================

what's the weather in sf?
================================== Ai Message ==================================

[{'id': 'toolu_01Dez1sJre4oA2Y7NsKJV6VT', 'input': {'query': 'weather in san francisco'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}]
Tool Calls:
  tavily_search_results_json (toolu_01Dez1sJre4oA2Y7NsKJV6VT)
 Call ID: toolu_01Dez1sJre4oA2Y7NsKJV6VT
  Args:
    query: weather in san francisco
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.accuweather.com/en/us/san-francisco/94103/weather-forecast/347629", "content": "Get the current and future weather conditions for San Francisco, CA, including temperature, precipitation, wind, air quality and more. See the hourly and 10-day outlook, radar maps, alerts and allergy information."}]
================================== Ai Message ======================